In [6]:
from matplotlib.pyplot import imread

# -*- coding: utf-8 -*-
"""
Created on Saturday Nov 15 15:34 2025

@author: 100yearsahead


Bleaching Presence Detection
Target variable: Percent_Bleaching

"""

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import os


path = "../../coral-reef-global-bleaching"
filename_read = os.path.join(path, "coral_whole.csv")

df = pd.read_csv(filename_read)

# Removed only locations and labels, no real data touched
# df.drop("Ocean_Name", axis=1, inplace=True)
# df.drop("Country_Name", axis=1, inplace=True)
# df.drop("Sample_ID", axis=1, inplace=True)
# df.drop("Date_Year", axis=1, inplace=True)
# df.drop("Bleaching_Level", axis=1, inplace=True)
# df.drop("Realm_Name", axis=1, inplace=True)
# # #Percent_Cover is not a best predictor and also contain 30% of its fields as null.
# # # For the sake of bigger dataset this feature is dropped
# df.drop("Percent_Cover", axis=1, inplace=True)
# # df.drop("ClimSST", inplace=True, axis=1)
# df.drop("Exposure", inplace=True, axis=1)
# # df.drop("Temperature_Maximum", inplace=True, axis=1)

# label_encoder = LabelEncoder()
# df["Exposure"] = label_encoder.fit_transform(df["Exposure"])

# These features were taken into account that data is nonlinear
# df = df[["Distance_to_Shore", "Temperature_Mean", "Turbidity", "TSA", "Depth_m", "Percent_Bleaching"]]

# These features were taken into account that data is linear
#df = df[['Cyclone_Frequency', 'Depth_m', 'ClimSST', 'Distance_to_Shore', 'Turbidity', 'TSA', 'Temperature_Mean', 'Percent_Bleaching']]

df.dropna(inplace=True)
df  = df.drop(columns=['Sample_ID'])
df.info()
print(df)


<class 'pandas.core.frame.DataFrame'>
Index: 22561 entries, 6981 to 35042
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Cyclone_Frequency    22561 non-null  float64
 1   Depth_m              22561 non-null  float64
 2   ClimSST              22561 non-null  float64
 3   Ocean_Name           22561 non-null  object 
 4   Country_Name         22561 non-null  object 
 5   Distance_to_Shore    22561 non-null  float64
 6   Exposure             22561 non-null  object 
 7   Turbidity            22561 non-null  float64
 8   Date_Year            22561 non-null  int64  
 9   Bleaching_Level      22561 non-null  object 
 10  Temperature_Maximum  22561 non-null  float64
 11  SSTA                 22561 non-null  float64
 12  TSA                  22561 non-null  float64
 13  Percent_Bleaching    22561 non-null  float64
 14  Temperature_Mean     22561 non-null  float64
 15  Realm_Name           22561 non-null  o

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder

# Split first (no leakage)
X = df.drop(columns=['Percent_Bleaching'])
y = df['Percent_Bleaching']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Since we have categorical variables we need to seperate the numeric and the categorical variables 
cat_cols = ['Realm_Name','Ocean_Name','Country_Name','Exposure','Bleaching_Level']
num_cols = [col for col in X.columns if col not in cat_cols]



# We one_hot_encode the categorical features
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

X_train_cat = ohe.fit_transform(X_train[cat_cols])
X_test_cat  = ohe.transform(X_test[cat_cols])


# We scale the numeric features
scaler = StandardScaler()

X_train_num = scaler.fit_transform(X_train[num_cols])
X_test_num  = scaler.transform(X_test[num_cols])


# Combine the categorical and numerical features
X_train_processed = np.hstack([X_train_num, X_train_cat])
X_test_processed  = np.hstack([X_test_num, X_test_cat])






In [8]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Dense, BatchNormalization, Dropout, ReLU, Add
)
from tensorflow.keras.callbacks import EarlyStopping


def build_mlp(input_dim):

    inputs = Input(shape=(input_dim,))
    # First block
    x = Dense(128)(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # Residual block  this allows for some of the inputs to skip layers which helps with vanishing gradients deeper learning we used
    skip = x
    r = Dense(128, activation='relu')(x)
    r = Dense(128)(r)
    x = Add()([skip, r])   # skip + residual
    x = ReLU()(x)

    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)

    outputs = Dense(1)(x)

    model = Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='mse',
        metrics=['mae']
    )

    return model

In [9]:
input_dim = X_train_processed.shape[1]
model = build_mlp(input_dim)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 104)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │     13,440 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 128)       │        512 │ dense[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 128)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │     16,512 │ re_lu[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 128)       │     16,512 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 128)       │          0 │ re_lu[0][0],      │
│                     │                   │            │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 128)       │          0 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 64)        │      8,256 │ re_lu_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 32)        │      2,080 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 1)         │         33 │ dense_4[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 57,345 (224.00 KB)

 Trainable params: 57,089 (223.00 KB)

 Non-trainable params: 256 (1.00 KB)

In [ ]:

# Stops training when the model stops improving prevents overfitting
early_stop = EarlyStopping(
    patience=10,
    restore_best_weights=True,
    monitor='loss'
)

history = model.fit(
    X_train_processed,
    y_train,
    validation_split=0.05,   # 20% of training becomes validation
    epochs=200,
    #batch_size=32,
    callbacks=[early_stop],
    verbose=2
)

Epoch 1/200
452/452 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 96.3906 - mae: 4.0603 - val_loss: 90.3422 - val_mae: 3.6384
Epoch 2/200
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 73.3396 - mae: 3.6427 - val_loss: 84.2783 - val_mae: 3.9290
Epoch 3/200
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 69.1998 - mae: 3.4536 - val_loss: 80.8979 - val_mae: 3.4284
Epoch 4/200
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 65.7860 - mae: 3.2981 - val_loss: 75.0963 - val_mae: 3.3740
Epoch 5/200
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 57.9240 - mae: 3.2054 - val_loss: 72.5969 - val_mae: 3.4579
Epoch 6/200
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 61.6192 - mae: 3.3046 - val_loss: 71.7713 - val_mae: 3.6205
Epoch 7/200
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 57.9350 - mae: 3.0726 - val_loss: 75.0358 - val_mae: 3.8916
Epoch 8/200
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 58.1411 - mae: 3.1007 - val_loss: 76.8344 - val_mae: 3.8797
Epoch 9/200
452/452 ━━━━━━━━━━━━

In [11]:
test_loss, test_mae = model.evaluate(X_test_processed, y_test, verbose=0)

print("Test MSE:", test_loss)
print("Test MAE:", test_mae)

Test MSE: 61.682682037353516
Test MAE: 3.361929416656494


In [12]:
y_pred = model.predict(X_test_processed)

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("R²:", r2)

142/142 ━━━━━━━━━━━━━━━━━━━━ 0s 803us/step
MSE: 61.682695083887
R²: 0.47551882333100504
